<a href="https://colab.research.google.com/github/oxanaRC/CE888/blob/master/CE888/Project3/tests2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# This is a very simple implementation of the UCT Monte Carlo Tree Search algorithm in Python 2.7.
# The function UCT(rootstate, itermax, verbose = False) is towards the bottom of the code.
# It aims to have the clearest and simplest possible code, and for the sake of clarity, the code
# is orders of magnitude less efficient than it could be made, particularly by using a 
# state.GetRandomMove() or state.DoRandomRollout() function.
# 
# Example GameState classes for Nim, OXO and Othello are included to give some idea of how you
# can write your own GameState use UCT in your 2-player game. Change the game to be played in 
# the UCTPlayGame() function at the bottom of the code.
# 
# Written by Peter Cowling, Ed Powley, Daniel Whitehouse (University of York, UK) September 2012.
# 
# Licence is granted to freely use and distribute for any sensible/legal purpose so long as this comment
# remains in any distributed code.
# 
# For more information about Monte Carlo Tree Search check out our web site at www.mcts.ai
#---------------------------------------------------------------------------------------------

In [0]:
from math import *
import random
import numpy as np
import csv
import pandas as pd 
import pickle
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split
# to save the model 
filename = 'saved_model.sav'
filenames=['saved_model0.sav','saved_model1.sav','saved_model2.sav', 'saved_model3.sav', 'saved_model4.sav', 'saved_model5.sav', 'saved_model6.sav', 'saved_model7.sav', 'saved_model8.sav', 'saved_model9.sav', 'saved_model10.sav' ]

In [0]:
class GameState:
    """ A state of the game, i.e. the game board. These are the only functions which are
        absolutely necessary to implement UCT in any 2-player complete information deterministic 
        zero-sum game, although they can be enhanced and made quicker, for example by using a 
        GetRandomMove() function to generate a random move during rollout.
        By convention the players are numbered 1 and 2.
    """
    def __init__(self):
            self.playerJustMoved = 2 # At the root pretend the player just moved is player 2 - player 1 has the first move

        
    def Clone(self):
        """ Create a deep clone of this game state.
        """
        st = GameState()
        st.playerJustMoved = self.playerJustMoved
        return st

    def DoMove(self, move):
        """ Update a state by carrying out the given move.
            Must update playerJustMoved.
        """
        self.playerJustMoved = 3 - self.playerJustMoved
        
    def GetMoves(self):
        """ Get all possible moves from this state.
        """
    
    def GetResult(self, playerjm):
        """ Get the game result from the viewpoint of playerjm. 
        """

    def __repr__(self):
        """ Don't need this - but good style.
        """
        pass


class OXOState:
    """ A state of the game, i.e. the game board.
        Squares in the board are in this arrangement
        012
        345
        678
        where 0 = empty, 1 = player 1 (X), 2 = player 2 (O)
    """
    def __init__(self):
        self.playerJustMoved = 2 # At the root pretend the player just moved is p2 - p1 has the first move
        self.board = [0,0,0,0,0,0,0,0,0] # 0 = empty, 1 = player 1, 2 = player 2
 
        
    def Clone(self):
        """ Create a deep clone of this game state.
        """
        st = OXOState()
        st.playerJustMoved = self.playerJustMoved
        st.board = self.board[:]
        return st

    def DoMove(self, move):
        """ Update a state by carrying out the given move.
            Must update playerToMove.
        """
        assert move >= 0 and move <= 8 and move == int(move) and self.board[move] == 0
        self.playerJustMoved = 3 - self.playerJustMoved
        self.board[move] = self.playerJustMoved
        
    def GetMoves(self):
        """ Get all possible moves from this state.
        """
        return [i for i in range(9) if self.board[i] == 0]
    
    def GetResult(self, playerjm):
        """ Get the game result from the viewpoint of playerjm. 
        """
        for (x,y,z) in [(0,1,2),(3,4,5),(6,7,8),(0,3,6),(1,4,7),(2,5,8),(0,4,8),(2,4,6)]:
            if self.board[x] == self.board[y] == self.board[z]:
                if self.board[x] == playerjm:
                    return 1.0
                else:
                    return 0.0
        if self.GetMoves() == []: return 0.5 # draw
        return False # Should not be possible to get here

    def __repr__(self):
        s= ""
        for i in range(9): 
            s += ".XO"[self.board[i]]
            if i % 3 == 2: s += "\n"
        return s
    
#----OC-170220-----
    def getStateData(self):
        myData =np.zeros([0,9], dtype=int)
        for i in range(9):
          myData = np.append(myData, self.board[i])
        #myData_str = np.array2string(myData, separator=', ')
        #myData_str=myData_str[1:-1] 
        #print (" state of the board: " + myData_str)
        #return myData_str 
        return myData
       
class Node:
    """ A node in the game tree. Note wins is always from the viewpoint of playerJustMoved.
        Crashes if state not specified.
    """
    def __init__(self, move = None, parent = None, state = None):
        self.move = move # the move that got us to this node - "None" for the root node
        self.parentNode = parent # "None" for the root node
        self.childNodes = []
        self.wins = 0
        self.visits = 0
        self.untriedMoves = state.GetMoves() # future child nodes
        self.playerJustMoved = state.playerJustMoved # the only part of the state that the Node needs later
        
    def UCTSelectChild(self):
        """ Use the UCB1 formula to select a child node. Often a constant UCTK is applied so we have
            lambda c: c.wins/c.visits + UCTK * sqrt(2*log(self.visits)/c.visits to vary the amount of
            exploration versus exploitation.
        """
        s = sorted(self.childNodes, key = lambda c: c.wins/c.visits + sqrt(2*log(self.visits)/c.visits))[-1]
        return s
    
    def AddChild(self, m, s):
        """ Remove m from untriedMoves and add a new child node for this move.
            Return the added child node
        """
        n = Node(move = m, parent = self, state = s)
        self.untriedMoves.remove(m)
        self.childNodes.append(n)
        return n
    
    def Update(self, result):
        """ Update this node - one additional visit and result additional wins. result must be from the viewpoint of playerJustmoved.
        """
        self.visits += 1
        self.wins += result

    def __repr__(self):
        return "[M:" + str(self.move) + " W/V:" + str(self.wins) + "/" + str(self.visits) + " U:" + str(self.untriedMoves) + "]"

    def TreeToString(self, indent):
        s = self.IndentString(indent) + str(self)
        for c in self.childNodes:
             s += c.TreeToString(indent+1)
        return s

    def IndentString(self,indent):
        s = "\n"
        for i in range (1,indent+1):
            s += "| "
        return s

    def ChildrenToString(self):
        s = ""
        for c in self.childNodes:
             s += str(c) + "\n"
        return s


def UCT( game_num, past_mode,rootstate, itermax ,learned_classifier, verbose = False):
    """ Conduct a UCT search for itermax iterations starting from rootstate.
        Return the best move from the rootstate.
        Assumes 2 alternating players (player 1 starts), with game results in the range [0.0, 1.0]."""

    rootnode = Node(state = rootstate)

    for i in range(itermax):
        node = rootnode
        state = rootstate.Clone()

        # Select
        while node.untriedMoves == [] and node.childNodes != []: # node is fully expanded and non-terminal
            node = node.UCTSelectChild()
            state.DoMove(node.move)

        # Expand
        if node.untriedMoves != []:  # if we can expand (i.e. state/node is non-terminal)
            m = random.choice(node.untriedMoves) 
            state.DoMove(m)
            node = node.AddChild(m, state)  # add child and descend tree

        # Rollout - this can often be made orders of magnitude quicker using a state.GetRandomMove() function
        while state.GetMoves() != []: # while state is non-terminal
            #---OC 130420--changes-----------------------------------------------------------
            if learned_classifier:
              player_moved_data=np.array([state.playerJustMoved])
              player_moved_data=player_moved_data.reshape((1, 1))
              states_data=np.array([state.getStateData()])
              states_data=states_data.reshape((1, -1))
              input_data=np.append(player_moved_data,states_data)
              input_data=input_data.reshape((1, -1))
              if past_mode==False: #OC -  we are collecting data and training the final model
                my_model = pickle.load(open(filename, 'rb'))
              else: #OC - we are doing the 10 test games
                my_model = pickle.load(open(filenames[game_num], 'rb'))
                #print('my model ' + str(filenames[game_num]))
              recommended_move = my_model.predict(input_data)
              #print ("----------------recommended move: " + str(recommended_move.item(0)) )
              if state.board[recommended_move.item(0)] == 0:
                 #print ('good recommended move, can have only these moves: ' + str(state.GetMoves()) )
                 state.DoMove(recommended_move.item(0))
              else:
                 #print ('bad recommended move, can have only these moves: ' + str(state.GetMoves()) + 'go random instead')
                 state.DoMove(random.choice(state.GetMoves())) 
            else: state.DoMove(random.choice(state.GetMoves()))

        # Backpropagate
        while node != None: # backpropagate from the expanded node and work back to the root node
            node.Update(state.GetResult(node.playerJustMoved)) # state is terminal. Update node with result from POV of node.playerJustMoved
            node = node.parentNode

    # Output some information about the tree - can be omitted
    if verbose: print(rootnode.TreeToString(0))
    #else: #print(rootnode.ChildrenToString())#OC 130420 commented printing to reduce output

    return sorted(rootnode.childNodes, key = lambda c: c.visits)[-1].move # return the move that was most visited

In [0]:
def UCTPlayGame(numberOfGame,df,df2,classifier, mode):
    """ Play a sample game between two UCT players where each player gets a different number 
        of UCT iterations (= simulations = tree nodes).
    """
    #------OC--130420----
    game_outcome=0 # 0 is the draw, 1 if player 1 won, 2 if player 2 won
    #--------------------
    state = OXOState() # uncomment to play OXO
    #------OC--100420-------------------------------------------
    data_writer=csv.writer(df, delimiter='\t',lineterminator='\n',)
    data_writer2=csv.writer(df2, delimiter='\t',lineterminator='\n',)
    if numberOfGame==0:     
       data_writer.writerow(['game_number, player_just_moved, cell0, cell1, cell2, cell3, cell4, cell5, cell6, cell7, cell8, best_move']) 
       data_writer2.writerow(['game_number, player_won'])
    #-----------------------------------------------------------    
    while state.GetMoves() != []:
        #print(str(state)) #OC commented printing out to reduce output
        if mode==False:  # this is not the final test
          if state.playerJustMoved == 1:         
              m = UCT(numberOfGame, mode, rootstate=state,itermax=1000,learned_classifier=classifier, verbose=False)  # play with values for itermax and verbose = True
          else:
              m = UCT(numberOfGame,mode,rootstate=state,itermax=100,learned_classifier=classifier, verbose=False) #OC note - changing intermax to 1000 here would results in player 1 winning all the time
        else:  #this is the final test
          if state.playerJustMoved == 1: 
              #player 2 is to use the latest model
              pastmode=False        
              m = UCT( numberOfGame,pastmode, rootstate=state,itermax=1000,learned_classifier=classifier, verbose=False)  # play with values for itermax and verbose = True
          else:
              #player 1 is to use previous models
              pastmode=True  
              m = UCT(numberOfGame,pastmode,rootstate=state,itermax=100, learned_classifier=classifier, verbose=False)
        
        #print("Best Move: " + str(m) + "\n") #OC commented printing out to reduce output
        #-----OC-170220-----
        state_str = np.array2string(state.getStateData(), separator=', ')
        state_str=state_str[1:-1]
        #data_writer.writerow([numberOfGame]+[',']+[state.playerJustMoved]+[',']+[state.getStateData()]+[',']+[m]) 
        data_writer.writerow([numberOfGame]+[',']+[state.playerJustMoved]+[',']+[state_str]+[',']+[m])
        #-------------------
        state.DoMove(m)
        if state.GetResult(state.playerJustMoved) != False:
            #print(str(state)) #OC commented printing out to reduce output
            break  
    if state.GetResult(state.playerJustMoved) == 1.0:
        #print("Player " + str(state.playerJustMoved) + " wins!") #OC commented printing out to reduce output
        #------OC--100420--to output number of the game and player who won---
        data_writer2.writerow([numberOfGame]+[',']+[state.playerJustMoved]) 
        game_outcome=state.playerJustMoved
    elif state.GetResult(state.playerJustMoved) == 0.0:
        #print("Player " + str(3 - state.playerJustMoved) + " wins!") #OC commented printing out to reduce output
        #------OC--100420--to output number of the game and player who won---
        data_writer2.writerow([numberOfGame]+[',']+[3-state.playerJustMoved]) 
        game_outcome=3-state.playerJustMoved
    #else:
        #print("Nobody wins!") #OC commented printing out to reduce output
    return game_outcome

In [0]:
def PlaySeveralGames(nGames,nCycle,firstCollection):
    n_player1wins=0
    n_player2wins=0
    n_draws=0
    counter=0
    n_games_random_classifier=0
    n_games_learned_classifier=0   
    test_run=false=False
    with open('data_file.csv',mode='w') as data_file:
       with open('data_filegames.csv',mode='w') as data_file2: 
         for gameNum in range(nGames):
           counter=counter+1
           if counter%10 ==0 or firstCollection:
              learned_classifier=False
              n_games_random_classifier=n_games_random_classifier+1
           else:
              learned_classifier=True
              n_games_learned_classifier=n_games_learned_classifier+1
           a_game=UCTPlayGame(gameNum,data_file,data_file2, learned_classifier, test_run) 
           if a_game==1:
             n_player1wins=n_player1wins+1
           if a_game==2:
             n_player2wins=n_player2wins+1  
           if a_game==0:  
             n_draws=n_draws+1
    print("Total games played " + str(n_games) + ". Number of draws:" +str(n_draws) + ". Number of wins by player 1: " + str(n_player1wins) + ". Number of wins by player 2: " + str(n_player2wins) )   
    print("Games played with learned classifier was " + str(n_games_learned_classifier) + ". Games played with random classifier was " + str(n_games_random_classifier)) 
    print('-----Next:-----------')

In [0]:
def PlayTestGame(numberG,data_file,data_file2):
  test_run=True
  print ("play game number " + str(numberG) + "player 1 is using model " + str(filenames[numberG]) + " player 2 is the latest model")
  learned_classifier=True 
  a_game=UCTPlayGame(numberG,data_file,data_file2, learned_classifier, test_run)  

In [0]:
def FitClassifier(firstCollection,cycle_number):
  df_states_and_moves = pd.read_csv("data_file.csv")
  df_won_games = pd.read_csv("data_filegames.csv")
  df=pd.merge(df_won_games, df_states_and_moves)
  if firstCollection:
     df.to_csv('data_merged.csv')
  else:
     df.to_csv('data_merged.csv', mode='a',header=False)    
  df_combined_merged = pd.read_csv("data_merged.csv", index_col=[0])
  df_combined_merged.drop_duplicates(subset=[' player_just_moved',' cell0',' cell1',' cell2',' cell3',' cell4',' cell5',' cell6',' cell7',' cell8'],inplace=True,keep='last')
  df_combined_merged.to_csv('data_merged.csv')
  dataset=np.loadtxt('data_merged.csv',delimiter=',',skiprows=1, dtype=int)
  #loading input data ('player just moved' and state of the cells ) columns into an array
  input_data=np.array(dataset[:,3:13], dtype=int)
  #print(input_data)
  result_data=np.array(dataset[:,[-1]])
  #print(result_data)
  result_data=np.array(dataset[:,[-1]]).reshape(result_data.shape[0])
  modelDT=DecisionTreeClassifier(criterion="entropy", max_depth=10)
  #x_train,x_test, y_train, y_test=train_test_split(input_data,result_data,test_size=0.2)
  #modelDT.fit(x_train,y_train)
  #y_pred=modelDT.predict(x_test)
  #score=metrics.accuracy_score(y_test,y_pred)
  #print(score)
  modelDT.fit(input_data,result_data)
  pickle.dump(modelDT, open(filename, 'wb'))
  pickle.dump(modelDT, open(filenames[cycle_number-1], 'wb'))#saving a model for each cycle


In [48]:
if __name__ == "__main__":
    """ Play a single game to the end using UCT for both players.

    """
    #------OC--100420--iterating over a number of games variable and collecting data for each game---
    n_games=20
    n_cycle=0
    first_data_collection=True
    PlaySeveralGames(n_games,n_cycle, first_data_collection) 
    FitClassifier(first_data_collection,n_cycle)     
    first_data_collection=False
    while n_cycle<10:
      n_cycle=n_cycle+1
      PlaySeveralGames(n_games,n_cycle, first_data_collection) 
      FitClassifier(first_data_collection,n_cycle ) 
    #play 10 games with all its past self
    test_game=0
    while test_game<=10:
      with open('data_file_test.csv',mode='w') as data_file:
        with open('data_filegames_test.csv',mode='w') as data_file2: 
          PlayTestGame(test_game,data_file,data_file2)
          df_states_and_moves_final_test = pd.read_csv("data_file_test.csv")
          if test_game==0:
            df_states_and_moves_final_test.to_csv('data_final_test.csv')
          else:
            df_states_and_moves_final_test.to_csv('data_final_test.csv', mode='a',header=False)    
          df_final = pd.read_csv("data_final_test.csv", index_col=[0])
          test_game=test_game+1
    display(df_final)     


Total games played 20. Number of draws:19. Number of wins by player 1: 1. Number of wins by player 2: 0
Games played with learned classifier was 0. Games played with random classifier was 20
-----Next:-----------
Total games played 20. Number of draws:18. Number of wins by player 1: 1. Number of wins by player 2: 1
Games played with learned classifier was 18. Games played with random classifier was 2
-----Next:-----------
Total games played 20. Number of draws:14. Number of wins by player 1: 2. Number of wins by player 2: 4
Games played with learned classifier was 18. Games played with random classifier was 2
-----Next:-----------
Total games played 20. Number of draws:20. Number of wins by player 1: 0. Number of wins by player 2: 0
Games played with learned classifier was 18. Games played with random classifier was 2
-----Next:-----------
Total games played 20. Number of draws:19. Number of wins by player 1: 1. Number of wins by player 2: 0
Games played with learned classifier was 18.

EmptyDataError: ignored

In [0]:
df_states_and_moves = pd.read_csv("data_file.csv")
df_won_games = pd.read_csv("data_filegames.csv")

In [0]:
df=pd.merge(df_won_games, df_states_and_moves)

In [0]:
display(df)

,game_number,player_won,player_just_moved,cell0,cell1,cell2,cell3,cell4,cell5,cell6,cell7,cell8,best_move
0,39,1,2,0,0,0,0,0,0,0,0,0,4
1,39,1,1,0,0,0,0,1,0,0,0,0,0
2,39,1,2,2,0,0,0,1,0,0,0,0,1
3,39,1,1,2,1,0,0,1,0,0,0,0,3
4,39,1,2,2,1,0,2,1,0,0,0,0,6
5,39,1,1,2,1,0,2,1,0,1,0,0,7
6,39,1,2,2,1,0,2,1,0,1,2,0,2
7,129,1,2,0,0,0,0,0,0,0,0,0,5
8,129,1,1,0,0,0,0,0,1,0,0,0,4
9,129,1,2,0,0,0,0,2,1,0,0,0,6


In [0]:
df.drop_duplicates(subset=[' player_just_moved',' cell0',' cell1',' cell2',' cell3',' cell4',' cell5',' cell6',' cell7',' cell8'],inplace=True,keep='last')

In [0]:
display(df)


,game_number,player_won,player_just_moved,cell0,cell1,cell2,cell3,cell4,cell5,cell6,cell7,cell8,best_move
4,39,1,2,2,1,0,2,1,0,0,0,0,6
5,39,1,1,2,1,0,2,1,0,1,0,0,7
6,39,1,2,2,1,0,2,1,0,1,2,0,2
8,129,1,1,0,0,0,0,0,1,0,0,0,4
9,129,1,2,0,0,0,0,2,1,0,0,0,6
10,129,1,1,0,0,0,0,2,1,1,0,0,0
11,129,1,2,2,0,0,0,2,1,1,0,0,8
12,129,1,1,2,0,0,0,2,1,1,0,1,1
13,129,1,2,2,2,0,0,2,1,1,0,1,2
15,190,1,1,0,0,0,0,1,0,0,0,0,0


In [0]:
df.to_csv('data_merged.csv')

In [0]:
dataset=np.loadtxt('data_merged.csv',delimiter=',',skiprows=1, dtype=int)
#loading input data ('player just moved' and state of the cells ) columns into an array
input_data=np.array(dataset[:,3:13], dtype=int)



In [0]:
input_data.shape

(80, 10)

In [0]:
input_data.shape[0]

80

In [0]:
#checking the output: first column has to show the player who moved and other columns show the state of the board
x=input_data[0:20,:]
x

array([[1, 1, 0, 0, 0, 2, 0, 0, 0, 1],
       [2, 1, 0, 0, 2, 2, 0, 0, 0, 1],
       [1, 1, 0, 0, 2, 2, 0, 0, 1, 1],
       [1, 2, 0, 0, 0, 0, 0, 0, 1, 1],
       [2, 2, 0, 0, 0, 0, 0, 2, 1, 1],
       [1, 2, 0, 0, 1, 0, 0, 2, 1, 1],
       [2, 2, 0, 2, 1, 0, 0, 2, 1, 1],
       [1, 2, 0, 2, 1, 1, 0, 2, 1, 1],
       [1, 2, 0, 0, 1, 0, 1, 0, 0, 0],
       [2, 2, 0, 0, 1, 2, 1, 0, 0, 0],
       [1, 2, 0, 0, 1, 2, 1, 0, 0, 1],
       [2, 2, 2, 0, 1, 2, 1, 0, 0, 1],
       [1, 2, 2, 0, 1, 2, 1, 0, 1, 1],
       [2, 0, 0, 0, 0, 2, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 2, 1, 1, 0, 0],
       [2, 2, 0, 0, 0, 2, 1, 1, 0, 0],
       [1, 2, 0, 0, 0, 2, 1, 1, 0, 1],
       [2, 2, 2, 0, 0, 2, 1, 1, 0, 1],
       [1, 2, 0, 0, 0, 0, 1, 1, 0, 0],
       [2, 2, 2, 0, 0, 0, 1, 1, 0, 0]])

In [0]:
#loading results data into an array (next best move)
result_data=np.array(dataset[:,[-1]])

In [0]:
result_data.shape

(80, 1)

In [0]:
#loading results data into an array 
result_data=np.array(dataset[:,[-1]]).reshape(result_data.shape[0])
print(result_data)

[3 7 5 6 3 2 4 1 4 8 1 7 2 6 0 8 1 2 1 2 4 8 4 2 0 8 2 1 6 3 4 0 2 3 6 4 5
 8 0 8 3 6 5 7 5 0 1 8 7 4 6 0 5 4 8 7 4 8 4 8 1 6 0 0 8 1 2 7 4 0 3 7 5 4
 0 1 3 6 7 2]


In [0]:
result_data.shape

(80,)

In [0]:
#splitting data into the training and testing sets
x_train,x_test, y_train, y_test=train_test_split(input_data,result_data,test_size=0.2)

In [0]:

modelDT=DecisionTreeClassifier(criterion="entropy", max_depth=10)
modelDT.fit(x_train,y_train)
y_pred=modelDT.predict(x_test)
score=metrics.accuracy_score(y_test,y_pred)

In [0]:
score

0.3125

In [0]:
from sklearn.svm import SVC


In [0]:
  modelSV2=SVC(kernel='poly', degree=2)
  modelSV2.fit(x_train,y_train)
  y_pred=modelSV2.predict(x_test)
  score=metrics.accuracy_score(y_test,y_pred)

In [0]:
score

0.3125

In [0]:

# save the model 
pickle.dump(modelDT, open(filename, 'wb'))

In [0]:
# load the model 
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(x_test,y_test)
print(result)

0.3125


In [0]:
if __name__ == "__main__":
    """ Play a single game to the end using UCT for both players.

    """
    #------OC--100420--iterating over a number of games variable and collecting data for each game---
    n_games=100
    n_player1wins=0
    n_player2wins=0
    n_draws=0
    counter=0
    n_games_random_classifier=0
    n_games_learned_classifier=0
    first_data_collection=False
    with open('data_file.csv',mode='w') as data_file:
       with open('data_filegames.csv',mode='w') as data_file2: 
         for gameNum in range(n_games):
           #print('NEW GAME ' + str(gameNum))   
           counter=counter+1
           if counter%10 ==0 or first_data_collection:
              learned_classifier=False
              n_games_random_classifier=n_games_random_classifier+1
           else:
              learned_classifier=True
              n_games_learned_classifier=n_games_learned_classifier+1
           a_game=UCTPlayGame(gameNum,data_file,data_file2, learned_classifier) 
           if a_game==1:
             n_player1wins=n_player1wins+1
           if a_game==2:
             n_player2wins=n_player2wins+1  
           if a_game==0:  
             n_draws=n_draws+1
    print("Total games played " + str(n_games) + ". Number of draws:" +str(n_draws) + ". Number of wins by player 1: " + str(n_player1wins) + ". Number of wins by player 2: " + str(n_player2wins) )   
    print("Games played with learned classifier was " + str(n_games_learned_classifier) + ". Games played with random classifier was " + str(n_games_random_classifier))           
    #---to do : train model and save the model---------------
    #---to do : play the game accessing the model and getting he predicted move from the model--

Total games played 100. Number of draws:96. Number of wins by player 1: 2. Number of wins by player 2: 2
Games played with learned classifier was 90. Games played with random classifier was 10


In [0]:
df_states_and_moves = pd.read_csv("data_file.csv")
df_won_games = pd.read_csv("data_filegames.csv")
df=pd.merge(df_won_games, df_states_and_moves)

In [0]:
display(df)

,game_number,player_won,player_just_moved,cell0,cell1,cell2,cell3,cell4,cell5,cell6,cell7,cell8,best_move
0,48,1,2,0,0,0,0,0,0,0,0,0,4
1,48,1,1,0,0,0,0,1,0,0,0,0,0
2,48,1,2,2,0,0,0,1,0,0,0,0,1
3,48,1,1,2,1,0,0,1,0,0,0,0,6
4,48,1,2,2,1,0,0,1,0,2,0,0,3
5,48,1,1,2,1,0,1,1,0,2,0,0,5
6,48,1,2,2,1,0,1,1,2,2,0,0,7
7,54,1,2,0,0,0,0,0,0,0,0,0,4
8,54,1,1,0,0,0,0,1,0,0,0,0,3
9,54,1,2,0,0,0,2,1,0,0,0,0,0


In [0]:
df.to_csv('data_merged.csv', mode='a',header=False)

In [0]:
df_combined_merged = pd.read_csv("data_merged.csv", index_col=[0])

In [0]:
df_combined_merged.drop_duplicates(subset=[' player_just_moved',' cell0',' cell1',' cell2',' cell3',' cell4',' cell5',' cell6',' cell7',' cell8'],inplace=True,keep='last')
#df_copy.drop_duplicates(subset=[" player_just_moved"," cell0"," cell1"],inplace=True,keep='last')
print (df_combined_merged)
df_combined_merged.to_csv('data_merged.csv')

    game_number   player_won   player_just_moved  ...   cell7   cell8   best_move
3             4            2                   1  ...       0       1           3
4             4            2                   2  ...       0       1           7
5             4            2                   1  ...       1       1           5
21           56            2                   1  ...       1       1           6
22           56            2                   2  ...       1       1           3
..          ...          ...                 ...  ...     ...     ...         ...
25           69            2                   1  ...       0       0           0
26           69            2                   2  ...       0       0           8
27           69            2                   1  ...       0       1           1
28           69            2                   2  ...       0       1           2
29           69            2                   1  ...       0       1           7

[88 rows x 13 c

In [0]:
dataset=np.loadtxt('data_merged.csv',delimiter=',',skiprows=1, dtype=int)
#loading input data ('player just moved' and state of the cells ) columns into an array
input_data=np.array(dataset[:,3:13], dtype=int)

In [0]:
input_data.shape

(88, 10)

In [0]:
#loading results data into an array (next best move)
result_data=np.array(dataset[:,[-1]])

In [0]:
#loading results data into an array 
result_data=np.array(dataset[:,[-1]]).reshape(result_data.shape[0])
print(result_data)

[3 7 5 6 3 2 4 1 4 8 1 7 2 6 0 8 1 2 1 2 4 8 4 2 0 8 2 1 6 3 4 0 2 3 6 4 5
 8 0 8 3 6 5 7 5 0 1 8 7 4 6 0 5 4 8 7 4 8 1 6 0 4 0 3 7 5 6 7 2 1 6 3 5 7
 3 0 8 1 7 2 3 4 6 0 8 1 2 7]


In [0]:
result_data.shape

(88,)

In [0]:
#loading results data into an array 
result_data=np.array(dataset[:,[-1]]).reshape(result_data.shape[0])
print(result_data)

[3 7 5 6 3 2 4 1 4 8 1 7 2 6 0 8 1 2 1 2 4 8 4 2 0 8 2 1 6 3 4 0 2 3 6 4 5
 8 0 8 3 6 5 7 5 0 1 8 7 4 6 0 5 4 8 7 4 8 1 6 0 4 0 3 7 5 6 7 2 1 6 3 5 7
 3 0 8 1 7 2 3 4 6 0 8 1 2 7]


In [0]:
result_data.shape

(88,)

In [0]:
#splitting data into the training and testing sets
x_train,x_test, y_train, y_test=train_test_split(input_data,result_data,test_size=0.2)

In [0]:
modelDT=DecisionTreeClassifier(criterion="entropy", max_depth=10)
modelDT.fit(x_train,y_train)
y_pred=modelDT.predict(x_test)
score=metrics.accuracy_score(y_test,y_pred)

In [0]:
score

0.16666666666666666

In [0]:
pickle.dump(modelDT, open(filename, 'wb'))

In [0]:
# load the model 
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(x_test,y_test)
print(result)

0.16666666666666666
